In [269]:
import numpy as np
import pandas as pd
import sqlite3
import math
import mgrs
from itertools import product
import statistics
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [261]:
def missingByCol(df): #count missing values by column
    for col in df.columns:
        if df[col].isna().sum() != 0:
            print(col, df[col].isna().sum())

def getCalTbl(start='2000-01-01', end='2015-12-31'):
    df = pd.DataFrame({'date': pd.date_range(start, end)})
    df['year'] = df.date.dt.year
    df['day_of_year'] = df.date.dt.dayofyear
    df['month'] = df.date.dt.month
    df['month_name'] = df.date.dt.month_name()
    df['day'] = df.date.dt.day
    df['day_of_week'] = df.date.dt.dayofweek
    df['day_of_week_name'] = df.date.dt.day_name()
    return df
    
'''
Run classification on data set with chosen clfs for each combination of hypers
Store results in a sorted list
Print all results
'''
def clfRun(a_clf, data, clf_hyper={}):
  M, L, n_folds = data # unpack data container
  kf = KFold(n_splits=n_folds) # Establish the cross validation
  ret = {} # classic explication of results
 
  for ids, (train_index, test_index) in enumerate(kf.split(M, L)):
       
    clf = a_clf(**clf_hyper) # unpack parameters into clf if they exist
 
    clf.fit(M[train_index], L[train_index])
 
    pred = clf.predict(M[test_index])
 
    ret[ids]= {'clf': clf,
              'train_index': train_index,
              'test_index': test_index,
              'accuracy': accuracy_score(L[test_index], pred)}
 
  return ret
 
def clfHypers(clfsList):
    ret_hyper = dict()
    for clf in clfsList:
        clfString = str(clf)  # Check if values in clfsList are in clf_hyperict
        for k1, v1 in clf_hyper.items():  # go through first level of clf_hyperict
            if k1 in clfString:            # if clfString1 matches first level
                ret_hyper[clf] = [dict(zip(v1, s))
                                  for s in product(*v1.values())]
    return ret_hyper
 
def clfGridSearch(X, y, n_folds):
      # suppress future warnings
      with warnings.catch_warnings():
            # ignore all caught warnings
            warnings.filterwarnings("ignore")
 
      # n_folds = 5
 
      data = (X, y, n_folds)


      hyper_param_dict = clfHypers(clfsList) 
 
      # run grid search to find best hyper params
      clfsAccuracyDict = {}
      results = {}
      for clfs in clfsList:
            for i in hyper_param_dict[clfs]:
                 
                  clf_hyper = i
 
                  results = clfRun(clfs, data, clf_hyper)
 
                  for key in results:
                        k = results[key]['clf']
                        v = results[key]['accuracy']
 
                        kTest = str(k)
 
                        kTest = kTest.replace('         ', ' ')
                        kTest = kTest.replace('        ', ' ')
 
                        if kTest in clfsAccuracyDict:
                             
                              clfsAccuracyDict[kTest].append(v)
                        else:
                              clfsAccuracyDict[kTest] = [v]
           
      clfsAccuracyList_sorted = sorted(clfsAccuracyDict.items(), key = lambda item: statistics.mean(item[1]), reverse = True)
 
      for i in clfsAccuracyList_sorted:
            print('\nClassifier with Parameters:', i[0], '\nMeanAccuracy', statistics.mean(i[1]))

def clfResults(clf, X_train, y_train, X_test, y_test):
 
    clfTmp = clf.fit(X_train, y_train) #fit model on training data
    y_pred = clfTmp.predict(X_test) #produce predictions (0/1) on test data
    y_pred_prob = clfTmp.predict_proba(X_test) #produce prediction probabilites (0 <= p <= 1) on test data
    y_pred_prob = y_pred_prob[:, 1] # only take second column.  first column is inverse prob
    tn, fp, fn, tp = confusion_matrix(y_test,y_pred).ravel() #get confusion matrix on test data
    total_obs = (tp + tn + fp + fn)
   # print(tn, fp, fn, tp)
 
 
    clfAcc = (tp + tn) / (tp + tn + fp + fn) # accuracy of entire model
    clfPrecision = tp / (tp + fp) # precision is the ability of the classifier not to label as positive a sample that is negative
    clfRecall = tp / (tp + fn) # recall is the ability of the classifier to find all the positive samples
 
    # return(clfAcc, clfPrecision, clfRecall)
    print('-----------------------Confusion Matrix------------------------'
        , '\n  Predicted|--------------------Actual------------------------'
        , '\n           |        Yes            |         No                |   Total   '
        , '\n--------------------------------------------------------------'
        , '\n        Yes|        '+ str(tp)+'            |        '+str(fp)+'               |    '+str(tp+fp)
        , '\n--------------------------------------------------------------'
        , '\n         No|        '+ str(fn)+'             |        '+str(tn)+'              |    '+str(fn+tn)
        , '\n--------------------------------------------------------------'
        , '\n      Total|        '+ str(tp+fn)+'            |        '+str(fp+tn)+'              |    '+str(total_obs)
        , '\n'
        )
    # print('Test Total Obs: ', total_obs)
    print('Accuracy: ', np.round(clfAcc,6), '\nPrecision: ', np.round(clfPrecision,6), '\nRecall: ', np.round(clfRecall,6))

Import Data: Need to copy and paste sqllite file from Kaggle into local drive path.
FPA_FOD_20170508.sqlite https://www.kaggle.com/rtatman/188-million-us-wildfires             
        
Metadata:   
Fires - Target data - Kaggle        
Time - Calendar data - Omits Dec through March (non fire season)      
Location - MGRS - Military Grid Ref. Sys. with lat/lon                      
Weather - USGS - Precip and Temp only (for now)

Get Fire Data

In [226]:
conn = sqlite3.connect('C:/Users/balso/Downloads/FPA_FOD_20170508.sqlite')
ca_fires = pd.read_sql_query('select * from fires where State = "CA";', conn) # 

m = mgrs.MGRS() #mgrs API
to_mgrs_1m = [] #precision 5
to_mgrs_10m = [] #precision 4
to_mgrs_100m = [] #precision 3
to_mgrs_1km = []  #precision 2
to_mgrs_10km = []  #precision 1
to_mgrs_100km = []  #precision 0
to_mgrs_lat_10km = []
to_mgrs_lon_10km = []
ca_fires_data = pd.DataFrame()

ca_fires_data['object_id'] = ca_fires['OBJECTID']
ca_fires_data['reporting_unit'] = ca_fires['NWCG_REPORTING_UNIT_NAME']
ca_fires_data['fire_date_num'] = ca_fires['DISCOVERY_DATE']
ca_fires_data['fire_date_cal'] = pd.to_datetime(ca_fires['DISCOVERY_DATE'], unit='D', origin='julian') #convert numeric to calendar date
ca_fires_data['fire_year'] = ca_fires['FIRE_YEAR']
ca_fires_data['fire_month'] = ca_fires_data['fire_date_cal'].dt.month_name()
ca_fires_data['fire_day_of_week'] = ca_fires_data['fire_date_cal'].dt.day_name()
ca_fires_data['fire_class'] = ca_fires['FIRE_SIZE_CLASS']
ca_fires_data['lat'] = ca_fires['LATITUDE']
ca_fires_data['lon'] = ca_fires['LONGITUDE']

for i, j in zip(ca_fires_data['lat'].tolist(), ca_fires_data['lon'].tolist()):
    to_mgrs_1m.append(m.toMGRS(i, j, MGRSPrecision=5)) # use mgrs API to convert lat/lon to MGRS conventions
    to_mgrs_10m.append(m.toMGRS(i, j, MGRSPrecision=4)) 
    to_mgrs_100m.append(m.toMGRS(i, j, MGRSPrecision=3)) 
    to_mgrs_1km.append(m.toMGRS(i, j, MGRSPrecision=2)) 
    to_mgrs_10km.append(m.toMGRS(i, j, MGRSPrecision=1)) 
    to_mgrs_100km.append(m.toMGRS(i, j, MGRSPrecision=0)) 

ca_fires_data['mgrs_1m'] = to_mgrs_1m
ca_fires_data['mgrs_10m'] = to_mgrs_10m
ca_fires_data['mgrs_100m'] = to_mgrs_100m
ca_fires_data['mgrs_1km'] = to_mgrs_1km
ca_fires_data['mgrs_10km'] = to_mgrs_10km
ca_fires_data['mgrs_100km'] = to_mgrs_100km

for k in ca_fires_data['mgrs_10km'].tolist():
    to_mgrs_lat_10km.append(m.toLatLon(k)[0])
    to_mgrs_lon_10km.append(m.toLatLon(k)[1])

ca_fires_data['mgrs_lat_10km'] = to_mgrs_lat_10km
ca_fires_data['mgrs_lon_10km'] = to_mgrs_lon_10km

# filter out winter months where fires not likely
months_used = ['April', 'May', 'June', 'July', 'August', 'September', 'October','November'] 
ca_fires_data = ca_fires_data[ca_fires_data.fire_month.isin(months_used)]

# filter out class A fires, which are less than 1/4 acre
classes_used = ['B','C','D','E','F','G']
ca_fires_data = ca_fires_data[ca_fires_data.fire_class.isin(classes_used)]

# filter out fires before 2000 since weather data is sparse before year 2000
ca_fires_data = ca_fires_data[ca_fires_data.fire_year >= 2000]

# limit to only the MGRS locations in Southern CA
mgrs_100km = ['11SMT','11SNT','11SNS','11SMS','11SLT'] # redefine mgrs blocks
socal_fires_data = ca_fires_data[ca_fires_data.mgrs_100km.isin(mgrs_100km)]

# keep only necessary columns
socal_lite = pd.DataFrame()
socal_lite['mgrs_100km'] = socal_fires_data['mgrs_100km']
socal_lite['mgrs_10km'] = socal_fires_data['mgrs_10km']
socal_lite['lat'] = socal_fires_data['mgrs_lat_10km']
socal_lite['lon'] = socal_fires_data['mgrs_lon_10km']
socal_lite['date'] = socal_fires_data['fire_date_cal']
socal_lite = socal_lite.groupby(level=0).first()
# socal_lite.mgrs_10km.unique().shape
socal_lite.reset_index(drop=True, inplace = True)
socal_lite.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8057 entries, 0 to 8056
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   mgrs_100km  8057 non-null   object        
 1   mgrs_10km   8057 non-null   object        
 2   lat         8057 non-null   float64       
 3   lon         8057 non-null   float64       
 4   date        8057 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 314.9+ KB


Get Calendar Data

In [227]:
cal_table = getCalTbl()
cal_table = cal_table[cal_table.month_name.isin(months_used)] # filter out winter months
cal_table['key'] = int(0) # dummy key for cross join later
cal_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3904 entries, 91 to 5812
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              3904 non-null   datetime64[ns]
 1   year              3904 non-null   int64         
 2   day_of_year       3904 non-null   int64         
 3   month             3904 non-null   int64         
 4   month_name        3904 non-null   object        
 5   day               3904 non-null   int64         
 6   day_of_week       3904 non-null   int64         
 7   day_of_week_name  3904 non-null   object        
 8   key               3904 non-null   int64         
dtypes: datetime64[ns](1), int64(6), object(2)
memory usage: 305.0+ KB


Get Weather Data and Impute Missing Values

In [228]:
usgs_weather = pd.read_csv('usgs_weather.csv')
usgs_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38204 entries, 0 to 38203
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   STATION    38204 non-null  object 
 1   NAME       38204 non-null  object 
 2   LATITUDE   38204 non-null  float64
 3   LONGITUDE  38204 non-null  float64
 4   ELEVATION  38204 non-null  float64
 5   DATE       38204 non-null  object 
 6   PRCP       37863 non-null  float64
 7   TMAX       36571 non-null  float64
 8   TMIN       36570 non-null  float64
 9   TOBS       24269 non-null  float64
dtypes: float64(7), object(3)
memory usage: 2.9+ MB


In [229]:
prcp_temp = pd.DataFrame()
prcp_temp['station_name'] = usgs_weather['NAME']
prcp_temp['lat'] = usgs_weather['LATITUDE']
prcp_temp['lon'] = usgs_weather['LONGITUDE']
prcp_temp['date'] = pd.to_datetime(usgs_weather['DATE'])
prcp_temp['prcp'] = usgs_weather['PRCP']
prcp_temp['tmax'] = usgs_weather['TMAX']
prcp_temp['tmin'] = usgs_weather['TMIN']
prcp_temp = prcp_temp[prcp_temp.date.dt.month_name().isin(months_used)]
prcp_temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25294 entries, 91 to 38203
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   station_name  25294 non-null  object        
 1   lat           25294 non-null  float64       
 2   lon           25294 non-null  float64       
 3   date          25294 non-null  datetime64[ns]
 4   prcp          25082 non-null  float64       
 5   tmax          24180 non-null  float64       
 6   tmin          24195 non-null  float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 1.5+ MB


In [230]:
df_hemet = cal_table.merge(prcp_temp[prcp_temp.station_name == 'HEMET, CA US'], on='date', how = 'left')
df_cuymaca = cal_table.merge(prcp_temp[prcp_temp.station_name == 'CUYAMACA, CA US'], on='date', how = 'left')
df_la = cal_table.merge(prcp_temp[prcp_temp.station_name == 'LOS ANGELES DOWNTOWN USC, CA US'], on='date', how = 'left')
df_elsinore = cal_table.merge(prcp_temp[prcp_temp.station_name == 'ELSINORE, CA US'], on='date', how = 'left')
df_vista = cal_table.merge(prcp_temp[prcp_temp.station_name == 'VISTA, CA US'], on='date', how = 'left')
print(df_hemet.shape, df_cuymaca.shape, df_la.shape, df_elsinore.shape, df_vista.shape)

df_hemet['station_name'] = df_hemet['station_name'].fillna('HEMET, CA US')
df_cuymaca['station_name'] = df_cuymaca['station_name'].fillna('CUYAMACA, CA US')
df_la['station_name'] = df_la['station_name'].fillna('LOS ANGELES DOWNTOWN USC, CA US')
df_elsinore['station_name'] = df_elsinore['station_name'].fillna('ELSINORE, CA US')
df_vista['station_name'] = df_vista['station_name'].fillna('VISTA, CA US')

df_hemet['lat'] = df_hemet['lat'].fillna(33.7381)
df_cuymaca['lat'] = df_cuymaca['lat'].fillna(32.9897)
df_la['lat'] = df_la['lat'].fillna(33.686)
df_elsinore['lat'] = df_elsinore['lat'].fillna(34.0236)
df_vista['lat'] = df_vista['lat'].fillna(33.2354)

df_hemet['lon'] = df_hemet['lon'].fillna(-116.8939)
df_cuymaca['lon'] = df_cuymaca['lon'].fillna(-116.5872)
df_la['lon'] = df_la['lon'].fillna(-117.3458)
df_elsinore['lon'] = df_elsinore['lon'].fillna(-118.2911)
df_vista['lon'] = df_vista['lon'].fillna(-117.2322)

data = pd.concat([df_hemet, df_cuymaca, df_elsinore, df_la, df_vista])
data.reset_index(drop=True, inplace=True)
print(data.shape)

(3904, 15) (3904, 15) (3904, 15) (3904, 15) (3904, 15)
(19520, 15)


In [231]:
missingByCol(data)

prcp 587
tmax 1354
tmin 1338


In [232]:
weather_nulls = data[data.isna().any(axis=1)]
weather_nulls

,date,year,day_of_year,month,month_name,day,day_of_week,day_of_week_name,key,station_name,lat,lon,prcp,tmax,tmin
494,2002-04-07,2002,97,4,April,7,6,Sunday,0,"HEMET, CA US",33.7381,-116.8939,0.0,73.0,NaN
1433,2005-10-31,2005,304,10,October,31,0,Monday,0,"HEMET, CA US",33.7381,-116.8939,0.0,NaN,48.0
1517,2006-05-24,2006,144,5,May,24,2,Wednesday,0,"HEMET, CA US",33.7381,-116.8939,0.0,94.0,NaN
1524,2006-05-31,2006,151,5,May,31,2,Wednesday,0,"HEMET, CA US",33.7381,-116.8939,0.0,NaN,NaN
2046,2008-07-04,2008,186,7,July,4,4,Friday,0,"HEMET, CA US",33.7381,-116.8939,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18544,2012-04-01,2012,92,4,April,1,6,Sunday,0,"VISTA, CA US",33.2354,-117.2322,NaN,NaN,NaN
18545,2012-04-02,2012,93,4,April,2,0,Monday,0,"VISTA, CA US",33.2354,-117.2322,NaN,NaN,NaN
18546,2012-04-03,2012,94,4,April,3,1,Tuesday,0,"VISTA, CA US",33.2354,-117.2322,NaN,NaN,NaN
18568,2012-04-25,2012,116,4,April,25,2,Wednesday,0,"VISTA, CA US",33.2354,-117.2322,NaN,NaN,NaN


In [233]:
data_missing = data.copy()
data_missing.drop(['date','year','day_of_year','month','month_name','day','day_of_week','day_of_week_name','key','station_name'], axis = 1, inplace = True)
imputer = KNNImputer(n_neighbors=2)
data_imputed = pd.DataFrame(imputer.fit_transform(data_missing), columns = data_missing.columns) # impute missing data here before the merge
missingByCol(data_imputed) # check for missing values

In [234]:
data_imputed.loc[494]

lat      33.7381
lon    -116.8939
prcp      0.0000
tmax     73.0000
tmin     44.5000
Name: 494, dtype: float64

In [235]:
print(len(data_imputed))

data.drop(['lat','lon','prcp','tmax','tmin'], axis = 1, inplace = True)
data['lat'] = data_imputed['lat']
data['lon'] = data_imputed['lon']
data['prcp'] = data_imputed['prcp']
data['tmax'] = data_imputed['tmax']
data['tmin'] = data_imputed['tmin']
missingByCol(data) # check for missing values

19520


In [236]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19520 entries, 0 to 19519
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              19520 non-null  datetime64[ns]
 1   year              19520 non-null  int64         
 2   day_of_year       19520 non-null  int64         
 3   month             19520 non-null  int64         
 4   month_name        19520 non-null  object        
 5   day               19520 non-null  int64         
 6   day_of_week       19520 non-null  int64         
 7   day_of_week_name  19520 non-null  object        
 8   key               19520 non-null  int64         
 9   station_name      19520 non-null  object        
 10  lat               19520 non-null  float64       
 11  lon               19520 non-null  float64       
 12  prcp              19520 non-null  float64       
 13  tmax              19520 non-null  float64       
 14  tmin              1952

In [237]:
data

,date,year,day_of_year,month,month_name,day,day_of_week,day_of_week_name,key,station_name,lat,lon,prcp,tmax,tmin
0,2000-04-01,2000,92,4,April,1,5,Saturday,0,"HEMET, CA US",33.7381,-116.8939,0.00,80.0,44.0
1,2000-04-02,2000,93,4,April,2,6,Sunday,0,"HEMET, CA US",33.7381,-116.8939,0.00,82.0,44.0
2,2000-04-03,2000,94,4,April,3,0,Monday,0,"HEMET, CA US",33.7381,-116.8939,0.00,87.0,43.0
3,2000-04-04,2000,95,4,April,4,1,Tuesday,0,"HEMET, CA US",33.7381,-116.8939,0.00,91.0,41.0
4,2000-04-05,2000,96,4,April,5,2,Wednesday,0,"HEMET, CA US",33.7381,-116.8939,0.00,86.0,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19515,2015-11-26,2015,330,11,November,26,3,Thursday,0,"VISTA, CA US",33.2354,-117.2322,0.00,62.0,45.0
19516,2015-11-27,2015,331,11,November,27,4,Friday,0,"VISTA, CA US",33.2354,-117.2322,0.51,61.0,46.0
19517,2015-11-28,2015,332,11,November,28,5,Saturday,0,"VISTA, CA US",33.2354,-117.2322,0.00,65.0,40.0
19518,2015-11-29,2015,333,11,November,29,6,Sunday,0,"VISTA, CA US",33.2354,-117.2322,0.00,67.0,44.0


In [238]:
# rolling averages for prcp, tmax, tmin for 3, 7, and 10 days

#prcp
data['prcp_rolling_3day'] = data.groupby('station_name')['prcp'].transform(lambda x: x.rolling(3).mean())
data['prcp_rolling_3day'] = round(data['prcp_rolling_3day'],6)

data['prcp_rolling_7day'] = data.groupby('station_name')['prcp'].transform(lambda x: x.rolling(7).mean())
data['prcp_rolling_7day'] = round(data['prcp_rolling_7day'],6)

data['prcp_rolling_14day'] = data.groupby('station_name')['prcp'].transform(lambda x: x.rolling(14).mean())
data['prcp_rolling_14day'] = round(data['prcp_rolling_14day'],6)

#tmax
data['tmax_rolling_3day'] = data.groupby('station_name')['tmax'].transform(lambda x: x.rolling(3).mean())
data['tmax_rolling_3day'] = round(data['tmax_rolling_3day'],6)

data['tmax_rolling_7day'] = data.groupby('station_name')['tmax'].transform(lambda x: x.rolling(7).mean())
data['tmax_rolling_7day'] = round(data['tmax_rolling_7day'],6)

data['tmax_rolling_14day'] = data.groupby('station_name')['tmax'].transform(lambda x: x.rolling(14).mean())
data['tmax_rolling_14day'] = round(data['tmax_rolling_14day'],6)

#tmin
data['tmin_rolling_3day'] = data.groupby('station_name')['tmin'].transform(lambda x: x.rolling(3).mean())
data['tmin_rolling_3day'] = round(data['tmin_rolling_3day'],6)

data['tmin_rolling_7day'] = data.groupby('station_name')['tmin'].transform(lambda x: x.rolling(7).mean())
data['tmin_rolling_7day'] = round(data['tmin_rolling_7day'],6)

data['tmin_rolling_14day'] = data.groupby('station_name')['tmin'].transform(lambda x: x.rolling(14).mean())
data['tmin_rolling_14day'] = round(data['tmin_rolling_14day'],6)

In [239]:
missingByCol(data)

prcp_rolling_3day 10
prcp_rolling_7day 30
prcp_rolling_14day 65
tmax_rolling_3day 10
tmax_rolling_7day 30
tmax_rolling_14day 65
tmin_rolling_3day 10
tmin_rolling_7day 30
tmin_rolling_14day 65


In [240]:
data_missing2 = data.copy()
data_missing2.drop(['date','year','day_of_year','month','month_name','day','day_of_week','day_of_week_name','key','station_name'], axis = 1, inplace = True)
imputer2 = KNNImputer(n_neighbors=2)
data_imputed2 = pd.DataFrame(imputer2.fit_transform(data_missing2), columns = data_missing2.columns) # impute missing data here before the merge
missingByCol(data_imputed2) # check for missing values

In [241]:
data_imputed2

,lat,lon,prcp,tmax,tmin,prcp_rolling_3day,prcp_rolling_7day,prcp_rolling_14day,tmax_rolling_3day,tmax_rolling_7day,tmax_rolling_14day,tmin_rolling_3day,tmin_rolling_7day,tmin_rolling_14day
0,33.7381,-116.8939,0.00,80.0,44.0,0.006666,0.002857,0.017857,77.500000,75.500000,78.178572,43.333333,43.500000,45.142858
1,33.7381,-116.8939,0.00,82.0,44.0,0.000000,0.039286,0.022857,81.000000,76.285714,72.428571,43.500000,43.000000,44.607143
2,33.7381,-116.8939,0.00,87.0,43.0,0.000000,0.000000,0.011428,83.000000,78.214286,74.821428,43.666667,44.071428,43.464286
3,33.7381,-116.8939,0.00,91.0,41.0,0.000000,0.000000,0.001429,86.666667,79.785714,82.071429,42.666667,43.857143,44.928572
4,33.7381,-116.8939,0.00,86.0,45.0,0.000000,0.000000,0.000000,88.000000,85.214286,79.571428,43.000000,42.500000,42.107142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19515,33.2354,-117.2322,0.00,62.0,45.0,0.036667,0.015714,0.010714,64.333333,75.285714,74.357143,47.333333,50.714286,48.428571
19516,33.2354,-117.2322,0.51,61.0,46.0,0.206667,0.088571,0.047143,61.333333,72.571429,73.214286,47.000000,50.142857,48.285714
19517,33.2354,-117.2322,0.00,65.0,40.0,0.170000,0.088571,0.047143,62.666667,69.285714,72.428571,43.666667,48.000000,47.857143
19518,33.2354,-117.2322,0.00,67.0,44.0,0.170000,0.088571,0.047143,64.333333,66.571429,72.285714,43.333333,46.000000,47.500000


In [242]:
print(len(data_imputed2))

data.drop(['prcp_rolling_3day','prcp_rolling_7day','prcp_rolling_14day','tmax_rolling_3day','tmax_rolling_7day','tmax_rolling_14day','tmin_rolling_3day','tmin_rolling_7day','tmin_rolling_14day'], axis = 1, inplace = True)
data['prcp_rolling_3day'] = data_imputed2['prcp_rolling_3day']
data['prcp_rolling_7day'] = data_imputed2['prcp_rolling_7day']
data['prcp_rolling_14day'] = data_imputed2['prcp_rolling_14day']
data['tmax_rolling_3day'] = data_imputed2['tmax_rolling_3day']
data['tmax_rolling_7day'] = data_imputed2['tmax_rolling_7day']
data['tmax_rolling_14day'] = data_imputed2['tmax_rolling_14day']
data['tmin_rolling_3day'] = data_imputed2['tmin_rolling_3day']
data['tmin_rolling_7day'] = data_imputed2['tmin_rolling_7day']
data['tmin_rolling_14day'] = data_imputed2['tmin_rolling_14day']
missingByCol(data) # check for missing values

19520


In [243]:
to_mgrs_10km, to_mgrs_100km = [],[] # get MGRS for each weather feed (5, 1 for each 100 km block)
w_lat = data['lat'].tolist()
w_lon = data['lon'].tolist()
for i, j in zip(w_lat, w_lon):
    to_mgrs_10km.append(m.toMGRS(i, j, MGRSPrecision=1)) 
    to_mgrs_100km.append(m.toMGRS(i, j, MGRSPrecision=0)) 
data['mgrs_100km'] = to_mgrs_100km # 
data['mgrs_10km'] = to_mgrs_10km

In [244]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19520 entries, 0 to 19519
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                19520 non-null  datetime64[ns]
 1   year                19520 non-null  int64         
 2   day_of_year         19520 non-null  int64         
 3   month               19520 non-null  int64         
 4   month_name          19520 non-null  object        
 5   day                 19520 non-null  int64         
 6   day_of_week         19520 non-null  int64         
 7   day_of_week_name    19520 non-null  object        
 8   key                 19520 non-null  int64         
 9   station_name        19520 non-null  object        
 10  lat                 19520 non-null  float64       
 11  lon                 19520 non-null  float64       
 12  prcp                19520 non-null  float64       
 13  tmax                19520 non-null  float64   

In [245]:
data = data[['date', 'day_of_year' ,'month_name' ,'day' ,'day_of_week' ,'day_of_week_name' ,'station_name', 'mgrs_100km' ,'lat' ,'lon' ,'prcp' ,'prcp_rolling_3day' ,'prcp_rolling_7day' ,'prcp_rolling_14day' ,'tmin' ,'tmin_rolling_3day','tmin_rolling_7day','tmin_rolling_14day','tmax','tmax_rolling_3day','tmax_rolling_7day','tmax_rolling_14day']]

In [246]:
data

,date,day_of_year,month_name,day,day_of_week,day_of_week_name,station_name,mgrs_100km,lat,lon,...,prcp_rolling_7day,prcp_rolling_14day,tmin,tmin_rolling_3day,tmin_rolling_7day,tmin_rolling_14day,tmax,tmax_rolling_3day,tmax_rolling_7day,tmax_rolling_14day
0,2000-04-01,92,April,1,5,Saturday,"HEMET, CA US",11SNT,33.7381,-116.8939,...,0.002857,0.017857,44.0,43.333333,43.500000,45.142858,80.0,77.500000,75.500000,78.178572
1,2000-04-02,93,April,2,6,Sunday,"HEMET, CA US",11SNT,33.7381,-116.8939,...,0.039286,0.022857,44.0,43.500000,43.000000,44.607143,82.0,81.000000,76.285714,72.428571
2,2000-04-03,94,April,3,0,Monday,"HEMET, CA US",11SNT,33.7381,-116.8939,...,0.000000,0.011428,43.0,43.666667,44.071428,43.464286,87.0,83.000000,78.214286,74.821428
3,2000-04-04,95,April,4,1,Tuesday,"HEMET, CA US",11SNT,33.7381,-116.8939,...,0.000000,0.001429,41.0,42.666667,43.857143,44.928572,91.0,86.666667,79.785714,82.071429
4,2000-04-05,96,April,5,2,Wednesday,"HEMET, CA US",11SNT,33.7381,-116.8939,...,0.000000,0.000000,45.0,43.000000,42.500000,42.107142,86.0,88.000000,85.214286,79.571428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19515,2015-11-26,330,November,26,3,Thursday,"VISTA, CA US",11SMS,33.2354,-117.2322,...,0.015714,0.010714,45.0,47.333333,50.714286,48.428571,62.0,64.333333,75.285714,74.357143
19516,2015-11-27,331,November,27,4,Friday,"VISTA, CA US",11SMS,33.2354,-117.2322,...,0.088571,0.047143,46.0,47.000000,50.142857,48.285714,61.0,61.333333,72.571429,73.214286
19517,2015-11-28,332,November,28,5,Saturday,"VISTA, CA US",11SMS,33.2354,-117.2322,...,0.088571,0.047143,40.0,43.666667,48.000000,47.857143,65.0,62.666667,69.285714,72.428571
19518,2015-11-29,333,November,29,6,Sunday,"VISTA, CA US",11SMS,33.2354,-117.2322,...,0.088571,0.047143,44.0,43.333333,46.000000,47.500000,67.0,64.333333,66.571429,72.285714


In [221]:
# data.to_csv('weather_data.csv')
socal_lite

,mgrs_100km,mgrs_10km,lat,lon,date
0,11SMT,11SMT27,34.067686,-117.866966,2005-07-03
1,11SMT,11SMT53,33.708802,-117.539594,2005-06-19
2,11SNS,11SNS16,33.078549,-116.892857,2005-07-31
3,11SNS,11SNS23,32.807799,-116.786366,2005-07-04
4,11SNS,11SNS08,33.258997,-117.000000,2005-07-05
...,...,...,...,...,...
8052,11SNS,11SNS12,32.717728,-116.893290,2015-04-09
8053,11SMT,11SMT95,33.890318,-117.108147,2009-05-03
8054,11SNS,11SNS04,32.898187,-117.000000,2010-05-29
8055,11SNS,11SNS04,32.898187,-117.000000,2014-05-29


In [247]:
data2 = data.merge(socal_lite, on = ('date', 'mgrs_100km'), how = 'left')

In [248]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22320 entries, 0 to 22319
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                22320 non-null  datetime64[ns]
 1   day_of_year         22320 non-null  int64         
 2   month_name          22320 non-null  object        
 3   day                 22320 non-null  int64         
 4   day_of_week         22320 non-null  int64         
 5   day_of_week_name    22320 non-null  object        
 6   station_name        22320 non-null  object        
 7   mgrs_100km          22320 non-null  object        
 8   lat_x               22320 non-null  float64       
 9   lon_x               22320 non-null  float64       
 10  prcp                22320 non-null  float64       
 11  prcp_rolling_3day   22320 non-null  float64       
 12  prcp_rolling_7day   22320 non-null  float64       
 13  prcp_rolling_14day  22320 non-null  float64   

In [249]:
data2

,date,day_of_year,month_name,day,day_of_week,day_of_week_name,station_name,mgrs_100km,lat_x,lon_x,...,tmin_rolling_3day,tmin_rolling_7day,tmin_rolling_14day,tmax,tmax_rolling_3day,tmax_rolling_7day,tmax_rolling_14day,mgrs_10km,lat_y,lon_y
0,2000-04-01,92,April,1,5,Saturday,"HEMET, CA US",11SNT,33.7381,-116.8939,...,43.333333,43.500000,45.142858,80.0,77.500000,75.500000,78.178572,11SNT72,33.617485,-116.24537
1,2000-04-02,93,April,2,6,Sunday,"HEMET, CA US",11SNT,33.7381,-116.8939,...,43.500000,43.000000,44.607143,82.0,81.000000,76.285714,72.428571,NaN,NaN,NaN
2,2000-04-03,94,April,3,0,Monday,"HEMET, CA US",11SNT,33.7381,-116.8939,...,43.666667,44.071428,43.464286,87.0,83.000000,78.214286,74.821428,NaN,NaN,NaN
3,2000-04-04,95,April,4,1,Tuesday,"HEMET, CA US",11SNT,33.7381,-116.8939,...,42.666667,43.857143,44.928572,91.0,86.666667,79.785714,82.071429,NaN,NaN,NaN
4,2000-04-05,96,April,5,2,Wednesday,"HEMET, CA US",11SNT,33.7381,-116.8939,...,43.000000,42.500000,42.107142,86.0,88.000000,85.214286,79.571428,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22315,2015-11-26,330,November,26,3,Thursday,"VISTA, CA US",11SMS,33.2354,-117.2322,...,47.333333,50.714286,48.428571,62.0,64.333333,75.285714,74.357143,NaN,NaN,NaN
22316,2015-11-27,331,November,27,4,Friday,"VISTA, CA US",11SMS,33.2354,-117.2322,...,47.000000,50.142857,48.285714,61.0,61.333333,72.571429,73.214286,NaN,NaN,NaN
22317,2015-11-28,332,November,28,5,Saturday,"VISTA, CA US",11SMS,33.2354,-117.2322,...,43.666667,48.000000,47.857143,65.0,62.666667,69.285714,72.428571,NaN,NaN,NaN
22318,2015-11-29,333,November,29,6,Sunday,"VISTA, CA US",11SMS,33.2354,-117.2322,...,43.333333,46.000000,47.500000,67.0,64.333333,66.571429,72.285714,NaN,NaN,NaN


In [250]:
data2.drop(['date', 'station_name', 'mgrs_100km', 'lat_x', 'lon_x', 'lat_y', 'lon_y'], axis = 1, inplace = True)

In [254]:
data2

,day_of_year,month_name,day,day_of_week,day_of_week_name,prcp,prcp_rolling_3day,prcp_rolling_7day,prcp_rolling_14day,tmin,tmin_rolling_3day,tmin_rolling_7day,tmin_rolling_14day,tmax,tmax_rolling_3day,tmax_rolling_7day,tmax_rolling_14day,is_fire
0,92,April,1,5,Saturday,0.00,0.006666,0.002857,0.017857,44.0,43.333333,43.500000,45.142858,80.0,77.500000,75.500000,78.178572,1
1,93,April,2,6,Sunday,0.00,0.000000,0.039286,0.022857,44.0,43.500000,43.000000,44.607143,82.0,81.000000,76.285714,72.428571,0
2,94,April,3,0,Monday,0.00,0.000000,0.000000,0.011428,43.0,43.666667,44.071428,43.464286,87.0,83.000000,78.214286,74.821428,0
3,95,April,4,1,Tuesday,0.00,0.000000,0.000000,0.001429,41.0,42.666667,43.857143,44.928572,91.0,86.666667,79.785714,82.071429,0
4,96,April,5,2,Wednesday,0.00,0.000000,0.000000,0.000000,45.0,43.000000,42.500000,42.107142,86.0,88.000000,85.214286,79.571428,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22315,330,November,26,3,Thursday,0.00,0.036667,0.015714,0.010714,45.0,47.333333,50.714286,48.428571,62.0,64.333333,75.285714,74.357143,0
22316,331,November,27,4,Friday,0.51,0.206667,0.088571,0.047143,46.0,47.000000,50.142857,48.285714,61.0,61.333333,72.571429,73.214286,0
22317,332,November,28,5,Saturday,0.00,0.170000,0.088571,0.047143,40.0,43.666667,48.000000,47.857143,65.0,62.666667,69.285714,72.428571,0
22318,333,November,29,6,Sunday,0.00,0.170000,0.088571,0.047143,44.0,43.333333,46.000000,47.500000,67.0,64.333333,66.571429,72.285714,0


In [253]:
data2['is_fire_bool'] = ~data2['mgrs_10km'].isna() # set NAs to 0
is_fire = []
for i in data2['is_fire_bool']:
    if i == False:
        is_fire.append(int(0))
    else:
        is_fire.append(int(1))

data2['is_fire'] = is_fire
data2.drop(['mgrs_10km','is_fire_bool'], axis = 1, inplace = True) # drop more columns

In [255]:
data2[data2['is_fire'] == 1] #why missing fires?

,day_of_year,month_name,day,day_of_week,day_of_week_name,prcp,prcp_rolling_3day,prcp_rolling_7day,prcp_rolling_14day,tmin,tmin_rolling_3day,tmin_rolling_7day,tmin_rolling_14day,tmax,tmax_rolling_3day,tmax_rolling_7day,tmax_rolling_14day,is_fire
0,92,April,1,5,Saturday,0.0,0.006666,0.002857,0.017857,44.0,43.333333,43.500000,45.142858,80.0,77.500000,75.500000,78.178572,1
24,116,April,25,1,Tuesday,0.0,0.000000,0.007143,0.042857,41.0,43.333333,42.571429,42.928571,94.0,84.666667,80.571429,78.571429,1
27,119,April,28,4,Friday,0.0,0.000000,0.000000,0.042857,47.0,49.666667,46.428571,43.857143,90.0,94.333333,86.428571,80.285714,1
28,120,April,29,5,Saturday,0.0,0.000000,0.000000,0.042857,43.0,46.666667,46.000000,43.642857,84.0,90.000000,88.714286,81.285714,1
34,126,May,5,4,Friday,0.0,0.000000,0.000000,0.000000,51.0,48.666667,46.571429,46.500000,89.0,90.666667,91.857143,89.142857,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22209,225,August,13,3,Thursday,0.0,0.000000,0.000000,0.000000,64.0,62.666667,63.857143,64.785714,86.0,81.666667,78.428571,80.428571,1
22219,235,August,23,6,Sunday,0.0,0.000000,0.000000,0.000000,68.0,67.666667,67.142857,65.357143,79.0,78.333333,79.142857,82.071429,1
22223,239,August,27,3,Thursday,0.0,0.000000,0.000000,0.000000,66.0,67.000000,67.142857,66.642857,92.0,87.333333,82.571429,83.642857,1
22260,276,October,3,5,Saturday,0.0,0.000000,0.000000,0.000000,60.0,60.666667,60.428571,63.285714,77.0,79.333333,82.142857,84.857143,1


Build Model

In [256]:
X = data2.copy()
# X

month_dummy = pd.get_dummies(X['month_name'])
wkname_dummy = pd.get_dummies(X['day_of_week_name'])
# mgrs_dummy = pd.get_dummies(X['mgrs_10km'])

X = pd.concat((X, month_dummy), axis = 1)
X = pd.concat((X, wkname_dummy), axis = 1)
# X = pd.concat((X, mgrs_dummy), axis = 1)

y = X['is_fire']

# X.drop(['month_name','day_of_week_name','mgrs_10km','is_fire'], axis = 1, inplace = True) # drop more columns
X.drop(['month_name','day_of_week_name','is_fire'], axis = 1, inplace = True) # drop more columns
print(X.shape, y.shape)

(22320, 30) (22320,)


In [257]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 8)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(17856, 30) (4464, 30) (17856,) (4464,)


In [271]:
clfsList = [LogisticRegression, RandomForestClassifier, GradientBoostingClassifier]

clf_hyper = { 
         'LogisticRegression':{
            'tol': [0.01, 0.1, 1.0]
           ,'C': [0.01, 0.1, 1.0]
           , 'solver': ['sag','saga']
           , 'class_weight': [None,'balanced']
        }
        ,
         'RandomForestClassifier':{
            'n_estimators': [50,100,150]
           ,'min_samples_leaf': [1,5,15]
           ,'class_weight': [None, 'balanced']
           ,'bootstrap': [True]
        }        
        ,
        'GradientBoostingClassifier':{
            'learning_rate': [0.001,0.01,0.1]
           ,'n_estimators': [10,50,100]
           ,'min_samples_leaf': [5,3,1]
        }
}

In [258]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

RandomForestClassifier()

In [272]:
clfGridSearch(np.array(X_train), np.array(y_train), 5)


Classifier with Parameters: RandomForestClassifier(n_estimators=150) 
MeanAccuracy 0.8138445885622736

Classifier with Parameters: RandomForestClassifier(class_weight='balanced') 
MeanAccuracy 0.813732245853825

Classifier with Parameters: RandomForestClassifier(class_weight='balanced', n_estimators=150) 
MeanAccuracy 0.8135086109549271

Classifier with Parameters: RandomForestClassifier() 
MeanAccuracy 0.8116602950920739

Classifier with Parameters: RandomForestClassifier(class_weight='balanced', n_estimators=50) 
MeanAccuracy 0.8106523779494077

Classifier with Parameters: RandomForestClassifier(n_estimators=50) 
MeanAccuracy 0.8090840329730945

Classifier with Parameters: RandomForestClassifier(min_samples_leaf=5) 
MeanAccuracy 0.7950829799463953

Classifier with Parameters: RandomForestClassifier(min_samples_leaf=5, n_estimators=150) 
MeanAccuracy 0.7949710919397673

Classifier with Parameters: RandomForestClassifier(min_samples_leaf=5, n_estimators=50) 
MeanAccuracy 0.79205897764

In [273]:
clf_1 = RandomForestClassifier(n_estimators=150)
clfResults(clf_1, X_train,y_train,X_test,y_test)

-----------------------Confusion Matrix------------------------ 
  Predicted|--------------------Actual------------------------ 
           |        Yes            |         No                |   Total    
-------------------------------------------------------------- 
        Yes|        1103            |        288               |    1391 
-------------------------------------------------------------- 
         No|        498             |        2575              |    3073 
-------------------------------------------------------------- 
      Total|        1601            |        2863              |    4464 

Accuracy:  0.823925 
Precision:  0.792955 
Recall:  0.688944


In [260]:
clfResults(clf, X_train, y_train, X_test, y_test)

-----------------------Confusion Matrix------------------------ 
  Predicted|--------------------Actual------------------------ 
           |        Yes            |         No                |   Total    
-------------------------------------------------------------- 
        Yes|        1095            |        297               |    1392 
-------------------------------------------------------------- 
         No|        506             |        2566              |    3072 
-------------------------------------------------------------- 
      Total|        1601            |        2863              |    4464 

Accuracy:  0.820116 
Precision:  0.786638 
Recall:  0.683948


In [274]:
importance1 = [clf_1.feature_importances_]

clf_1_importance = pd.DataFrame(importance1, columns = X_train.columns).T

clf_1_importance.columns = ['coeff']

clf_1_importance.sort_values(by = 'coeff', ascending = False)

,coeff
tmax_rolling_7day,0.112450
tmax_rolling_3day,0.107566
tmax_rolling_14day,0.103076
tmax,0.086948
tmin_rolling_14day,0.082719
day_of_year,0.080326
tmin_rolling_7day,0.074204
tmin_rolling_3day,0.067523
tmin,0.058982
day,0.054362
